In [10]:
import pandas as pd

In [ ]:
# Phil's code starts here

In [ ]:
# Phil's code ends here

In [2]:
# Jenna's code starts here

In [3]:
# Jenna's code ends here

In [4]:
# Ben's code starts here

In [5]:
# Ben's code ends here

In [6]:
# Katrina's code starts here

In [7]:
# Katrina's code ends here